In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from fastdtw import fastdtw
%matplotlib inline

In [2]:
def dateconverter(value): 
    s= str(value)
    Date=datetime.strptime(value, '%m/%d/%y')
    return Date

In [3]:
def get_countydict(df):
    Counties={}
    for value in df.County.unique():
        Counties[value]=df.loc[df['County'] == value]
    return Counties

This section is EDA and cleaning of all drought severity data dealing with California.

In [4]:
CA_SEV_DATA1=pd.read_csv('datasets/California_Drought_Severity_By_County_1997-2002.csv')
CA_SEV_DATA2=pd.read_csv('datasets/California_Drought_Severity_By_County_2002-2007.csv')
CA_SEV_DATA3=pd.read_csv('datasets/California_Drought_Severity_By_County_2007-2012.csv')
CA_SEV_DATA4=pd.read_csv('datasets/California_Drought_Severity_By_County_2012-2017.csv')

In [5]:
California_Severity_Data=pd.concat([CA_SEV_DATA1,CA_SEV_DATA2,CA_SEV_DATA3,CA_SEV_DATA4],axis=0)
California_Severity_Data.shape

(94219, 12)

In [6]:
California_Severity_Data.rename(columns={'Name': 'County'}, inplace=True)

In [7]:
CA_SEV_Dates=pd.DataFrame(California_Severity_Data.ReleaseDate.apply(dateconverter))

In [8]:
California_Severity_Data.ReleaseDate=CA_SEV_Dates.ReleaseDate

In [9]:
CA_years=[]
for value in California_Severity_Data['ReleaseDate']:
    CA_years.append(value.year)

In [10]:
CAyearsdf=pd.DataFrame(data=CA_years)

In [11]:
CAyearsdf.rename(columns={0: 'Years'}, inplace=True)

In [12]:
California_Severity_Data = California_Severity_Data.reset_index(drop=True)

In [13]:
California_Severity_Data['Year']=CAyearsdf['Years']

In [14]:
California_Severity_Data.sort_values(by='ReleaseDate',ascending=True,inplace=True)

In [15]:
California_Severity_Data.drop(['Id','PercentChangeFromWAve',
                             'AreaChangeFromWAve','StatisticFormatID'],axis=1,
                             inplace=True)

In [16]:
California=get_countydict(California_Severity_Data)

In [17]:
CABioregiondict={'Sierra County':'Sierra',
               'Amador County':'Sierra',
               'Placer County':'Sierra',
               'Alpine County':'Sierra',
               'Santa Clara County':'Bay/Delta',
               'Yolo County':'Sacramento Valley',
               'Mono County':'Sierra',
               'Ventura County':'Central Coast',
               'Modoc County':'Modoc',
               'Napa County':'Bay/Delta',
               'San Luis Obispo County':'Central Coast',
               'Los Angeles County':'South Coast',
               'Contra Costa County':'Bay/Delta',
               'Yuba County':'Sacramento Valley',
               'Solano County':'Bay/Delta', 
               'Del Norte County':'Klamath/North Coast',
               'Riverside County':'Mojave',
               'Monterey County':'Central Coast',
               'Plumas County':'Sierra',
               'San Francisco County':'Bay/Delta',
               'Mariposa County':'Sierra',
               'Shasta County':'Sacramento Valley',
               'Santa Cruz County':'Central Coast',
               'Tehama County':'Sacramento Valley',
               'Imperial County':'Colorado Desert',
               'Siskiyou County':'Klamath/North Coast',
               'El Dorado County':'Sierra',
               'Mendocino County':'Klamath/North Coast',
               'Fresno County':'San Joaquin Valley',
               'San Bernardino County':'Mojave',
               'San Joaquin County':'San Joaquin Valley',
               'Tulare County':'San Joaquin Valley',
               'Alameda County':'Bay/Delta',
               'Kings County':'San Joaquin Valley',
               'Butte County':'Sacramento Valley',
               'Merced County':'San Joaquin Valley',
               'Sacramento County':'Sacramento Valley',
               'Tuolumne County':'Sierra',
               'Sonoma County':'Bay/Delta',
               'Santa Barbara County':'Central Coast',
               'Madera County':'San Joaquin Valley',
               'Lassen County':'Modoc',
               'Glenn County':'Sacramento Valley',
               'Sutter County':'Sacramento Valley',
               'San Diego County':'South Coast',
               'Lake County':'Klamath/North Coast',
               'Nevada County':'Sierra',
               'Trinity County':'Klamath/North Coast',
               'Inyo County':'Sierra',
               'San Mateo County':'Bay/Delta',
               'Orange County':'South Coast',
               'Humboldt County':'Klamath/North Coast',
               'Colusa County':'Sacramento Valley',
               'Calaveras County':'Sierra',
               'Kern County':'San Joaquin Valley',
               'San Benito County':'Central Coast',
               'Marin County':'Bay/Delta',
               'Stanislaus County':'San Joaquin Valley'
              }
for key, value in CABioregiondict.items():
    California[key]['Bioregion']=value

/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
DTWvalues={}
for value in California.keys():
    if value=='Fresno County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Fresno County'].USDMLevelID)
        DTWvalues[value]=test_list[0]
    #test_list2=test_list[1:]
   # x = []; y=[]
    #for point in test_list2[0]:
       # x.append(point[0])
        #y.append(point[1])
    #plt.figure()
   # plt.scatter(x,y)

In [19]:
DTWvalues={}
for value in California.keys():
    if value=='Fresno County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Fresno County'].USDMLevelID)
        DTWvalues[value]=test_list[0]

In [20]:
DTWvalues2={}
for value in California.keys():
    if value=='Tulare County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Tulare County'].USDMLevelID)
        DTWvalues2[value]=test_list[0]

In [21]:
DTWvalues3={}
for value in California.keys():
    if value=='Monterey County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Monterey County'].USDMLevelID)
        DTWvalues3[value]=test_list[0]

'Merced County',
'Stanislaus County',
'San Joaquin County',
'Ventura County',
'',
'Imperial County'

In [22]:
#test_list2

In [23]:
DTWvalues4={}
for value in California.keys():
    if value=='Kern County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Kern County'].USDMLevelID)
        DTWvalues4[value]=test_list[0]

In [24]:
DTWvalues5={}
for value in California.keys():
    if value=='Merced County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Merced County'].USDMLevelID)
        DTWvalues5[value]=test_list[0]

In [25]:
DTWvalues6={}
for value in California.keys():
    if value=='Stanislaus County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Stanislaus County'].USDMLevelID)
        DTWvalues6[value]=test_list[0]
    

In [26]:
DTWvalues7={}
for value in California.keys():
    if value=='San Joaquin County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['San Joaquin County'].USDMLevelID)
        DTWvalues7[value]=test_list[0]

In [27]:
DTWvalues8={}
for value in California.keys():
    if value=='Ventura County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Ventura County'].USDMLevelID)
        DTWvalues8[value]=test_list[0]

In [28]:
Cali_Keys=California.keys()


In [29]:
Cali_Keys

dict_keys(['Alameda County', 'Monterey County', 'Napa County', 'Nevada County', 'Orange County', 'Placer County', 'Plumas County', 'Mono County', 'Riverside County', 'Alpine County', 'San Benito County', 'San Bernardino County', 'San Diego County', 'San Joaquin County', 'San Luis Obispo County', 'Sacramento County', 'Modoc County', 'Merced County', 'Mariposa County', 'Butte County', 'Calaveras County', 'Colusa County', 'Contra Costa County', 'El Dorado County', 'Fresno County', 'Glenn County', 'Imperial County', 'Inyo County', 'Kern County', 'Kings County', 'Lake County', 'Lassen County', 'Los Angeles County', 'Madera County', 'Santa Barbara County', 'Santa Clara County', 'Amador County', 'Solano County', 'Stanislaus County', 'Tehama County', 'Sutter County', 'Tuolumne County', 'Sonoma County', 'Ventura County', 'Yuba County', 'Yolo County', 'Tulare County', 'Mendocino County', 'Shasta County', 'Sierra County', 'Siskiyou County', 'Humboldt County', 'Trinity County', 'Del Norte County',

In [30]:
DTWvalues9={}
San_Diego=[]
for value in California.keys():
    DTWvalues9[value] = []
    test_list=fastdtw(California[value].USDMLevelID,
    California['San Diego County'].USDMLevelID)
    DTWvalues9[value].append(test_list[0])
    San_Diego.append(test_list[0])

In [31]:
San_Diego

[1520.0,
 1772.0,
 1503.0,
 1910.0,
 1658.0,
 1590.0,
 1599.0,
 1608.0,
 1603.0,
 1451.0,
 1413.0,
 1858.0,
 221.0,
 1746.0,
 2044.0,
 1355.0,
 1650.0,
 1619.0,
 1712.0,
 1258.0,
 1388.0,
 1254.0,
 1686.0,
 1378.0,
 1790.0,
 1296.0,
 1321.0,
 2044.0,
 1724.0,
 1519.0,
 995.0,
 1562.0,
 1689.0,
 1545.0,
 1677.0,
 1558.0,
 1234.0,
 1325.0,
 1528.0,
 1394.0,
 1603.0,
 1516.0,
 1102.0,
 1726.0,
 1261.0,
 1720.0,
 2001.0,
 1235.0,
 1396.0,
 1764.0,
 1356.0,
 1101.0,
 1251.0,
 1099.0,
 1353.0,
 1163.0,
 1132.0,
 1270.0]

In [32]:
DTWvalues10={}
for value in California.keys():
    if value=='Imperial County':
        continue
    else:
        test_list=fastdtw(California[value].USDMLevelID,
        California['Imperial County'].USDMLevelID)
        DTWvalues10[value]=test_list[0]


In [33]:
print('Fresno!')
print(max(DTWvalues, key=DTWvalues.get))
print(min(DTWvalues, key=DTWvalues.get))

Fresno!
Imperial County
Madera County


In [34]:
print('Tulare!')
print(max(DTWvalues2, key=DTWvalues2.get))
print(min(DTWvalues2, key=DTWvalues2.get))

Tulare!
San Diego County
Madera County


In [35]:
print('Monterey!')
print(max(DTWvalues3, key=DTWvalues3.get))
print(min(DTWvalues3, key=DTWvalues3.get))

Monterey!
Modoc County
Santa Clara County


In [36]:
print('Kern!')
print(max(DTWvalues4, key=DTWvalues4.get))
print(min(DTWvalues4, key=DTWvalues4.get))

Kern!
Modoc County
Tulare County


In [37]:
print('Merced!')
print(max(DTWvalues5, key=DTWvalues5.get))
print(min(DTWvalues5, key=DTWvalues5.get))

Merced!
Modoc County
Stanislaus County


In [38]:
print('Stanislaus!')
print(max(DTWvalues6, key=DTWvalues6.get))
print(min(DTWvalues6, key=DTWvalues6.get))

Stanislaus!
Modoc County
Merced County


In [39]:
print('San Joaquin!')
print(max(DTWvalues7, key=DTWvalues7.get))
print(min(DTWvalues7, key=DTWvalues7.get))

San Joaquin!
Modoc County
Sacramento County


In [40]:
print('Ventura!')
print(max(DTWvalues8, key=DTWvalues8.get))
print(min(DTWvalues8, key=DTWvalues8.get))

Ventura!
Modoc County
Mariposa County


In [41]:
print('San Diego!')
print(max(DTWvalues9, key=DTWvalues9.get))
print(min(DTWvalues9, key=DTWvalues9.get))

San Diego!
San Luis Obispo County
San Diego County


In [42]:
print('Imperial!')
print(max(DTWvalues10, key=DTWvalues10.get))
print(min(DTWvalues10, key=DTWvalues10.get))

Imperial!
Kern County
Del Norte County


In [43]:
DTWvalues8['Alameda County']

1220.0

In [44]:
DTWvalues

{'Alameda County': 1264.0,
 'Alpine County': 1392.0,
 'Amador County': 1225.0,
 'Butte County': 1213.0,
 'Calaveras County': 1267.0,
 'Colusa County': 1176.0,
 'Contra Costa County': 1210.0,
 'Del Norte County': 1492.0,
 'El Dorado County': 1389.0,
 'Glenn County': 1172.0,
 'Humboldt County': 1405.0,
 'Imperial County': 1944.0,
 'Inyo County': 1550.0,
 'Kern County': 1195.0,
 'Kings County': 1129.0,
 'Lake County': 1445.0,
 'Lassen County': 1419.0,
 'Los Angeles County': 1399.0,
 'Madera County': 1004.0,
 'Marin County': 1404.0,
 'Mariposa County': 1041.0,
 'Mendocino County': 1497.0,
 'Merced County': 1195.0,
 'Modoc County': 1799.0,
 'Mono County': 1472.0,
 'Monterey County': 1184.0,
 'Napa County': 1208.0,
 'Nevada County': 1303.0,
 'Orange County': 1235.0,
 'Placer County': 1164.0,
 'Plumas County': 1275.0,
 'Riverside County': 1354.0,
 'Sacramento County': 1148.0,
 'San Benito County': 1194.0,
 'San Bernardino County': 1396.0,
 'San Diego County': 1824.0,
 'San Francisco County': 

In [45]:
California_Severity_Data.head()

State  PercentCurrent AreaCurrent  USDMLevelID USDMLevel  \
0       CA           60.79      455.19            1      D0     
3337    CA           24.19      803.63            1      D0     
3373    CA           98.01      769.98            1      D0     
3432    CA           24.68      241.03            1      D0     
3616    CA          100.00      795.81            1      D0     

               County AreaMiles ReleaseDate  Year  
0      Alameda County    748.73  2000-01-04  2000  
3337  Monterey County  3,322.61  2000-01-04  2000  
3373      Napa County    785.58  2000-01-04  2000  
3432    Nevada County    976.61  2000-01-04  2000  
3616    Orange County    795.81  2000-01-04  2000

In [46]:
for idx,value in enumerate(California_Severity_Data.AreaCurrent):
    value=value.replace(',', '')
    California_Severity_Data.iloc[idx,2] = float(value)
California_Severity_Data.head()


State  PercentCurrent AreaCurrent  USDMLevelID USDMLevel  \
0       CA           60.79      455.19            1      D0     
3337    CA           24.19      803.63            1      D0     
3373    CA           98.01      769.98            1      D0     
3432    CA           24.68      241.03            1      D0     
3616    CA          100.00      795.81            1      D0     

               County AreaMiles ReleaseDate  Year  
0      Alameda County    748.73  2000-01-04  2000  
3337  Monterey County  3,322.61  2000-01-04  2000  
3373      Napa County    785.58  2000-01-04  2000  
3432    Nevada County    976.61  2000-01-04  2000  
3616    Orange County    795.81  2000-01-04  2000

In [47]:
for idx,value in enumerate(California_Severity_Data.AreaMiles):
    value=value.replace(',', '')
    California_Severity_Data.iloc[idx,6] = float(value)
California_Severity_Data.head()

State  PercentCurrent AreaCurrent  USDMLevelID USDMLevel  \
0       CA           60.79      455.19            1      D0     
3337    CA           24.19      803.63            1      D0     
3373    CA           98.01      769.98            1      D0     
3432    CA           24.68      241.03            1      D0     
3616    CA          100.00      795.81            1      D0     

               County AreaMiles ReleaseDate  Year  
0      Alameda County    748.73  2000-01-04  2000  
3337  Monterey County   3322.61  2000-01-04  2000  
3373      Napa County    785.58  2000-01-04  2000  
3432    Nevada County    976.61  2000-01-04  2000  
3616    Orange County    795.81  2000-01-04  2000

In [48]:
California_Severity_Data

State  PercentCurrent AreaCurrent  USDMLevelID USDMLevel  \
0        CA           60.79      455.19            1      D0     
3337     CA           24.19      803.63            1      D0     
3373     CA           98.01      769.98            1      D0     
3432     CA           24.68      241.03            1      D0     
3616     CA          100.00      795.81            1      D0     
3696     CA           30.84      462.71            1      D0     
3879     CA            0.97       25.23            1      D0     
3090     CA           80.96      2537.6            1      D0     
4055     CA           44.66     3266.17            1      D0     
55       CA           77.71      575.59            1      D0     
4258     CA           75.20     1046.14            1      D0     
4290     CA           14.03     2823.42            1      D0     
4428     CA           95.60     4070.14            1      D0     
4589     CA          100.00     1422.47            1      D0     
4665     CA           71.40     2360.12            1      D0     
4180     CA          100.00      989.02            1      D0     
2858     CA           44.17      1855.6            1      D0     
2784     CA          100.00     1968.56            1      D0     
2552     CA          100.00     1463.61            1      D0     
390      CA           93.53     1569.21            1      D0     
522      CA          100.00     1036.62            1      D0     
658      CA           97.15     1129.53            1      D0     
737      CA           86.72      635.21            1      D0     
951      CA           43.06      768.78            1      D0     
1133     CA           98.30     5913.33            1      D0     
1300     CA           81.10      1075.4            1      D0     
1517     CA           22.85     1030.37            1      D0     
1631     CA            2.72      278.44            1      D0     
1825     CA           93.84     7651.72            1      D0     
1937     CA          100.00     1387.02            1      D0     
...     ...             ...         ...          ...       ...   
92622    CA          100.00     1859.83            2      D1     
52054    CA           21.45     1290.14            1      D0     
60115    CA           53.64     2187.47            2      D1     
72542    CA            1.70      124.27            2      D1     
92623    CA          100.00     1859.83            1      D0     
76333    CA          100.00     4257.33            1      D0     
52055    CA           21.45     1290.14            1      D0     
54156    CA           99.77     4498.24            1      D0     
54157    CA           84.68     3817.95            2      D1     
56305    CA           38.73     3158.06            1      D0     
56306    CA            3.50       285.3            2      D1     
57355    CA           15.75      218.45            1      D0     
60117    CA          100.00     4078.02            1      D0     
60118    CA           53.64     2187.47            2      D1     
60119    CA            1.41       57.48            3      D2     
79028    CA            8.13      268.88            2      D1     
76334    CA           17.40      740.63            2      D1     
92624    CA          100.00     1859.83            2      D1     
67234    CA           25.95      862.12            1      D0     
69817    CA          100.00      795.81            1      D0     
69818    CA           99.11      788.73            2      D1     
72543    CA           53.43     3907.86            1      D0     
72544    CA            1.70      124.27            2      D1     
74409    CA           83.13     1156.39            1      D0     
75476    CA           13.92     2799.72            1      D0     
75477    CA            1.09      219.91            2      D1     
80804    CA           99.67     2741.27            2      D1     
80803    CA          100.00     2750.34            1      D0     
64372    CA            9.05      178.11       

In [62]:
CountyDummies=pd.get_dummies(California_Severity_Data.County)

In [67]:
California_Severity_Data=pd.concat([California_Severity_Data,CountyDummies],axis=1)
    

In [69]:
California_Severity_Data.columns

Index(['State', 'PercentCurrent', 'AreaCurrent', 'USDMLevelID', 'USDMLevel',
       'County', 'AreaMiles', 'ReleaseDate', 'Year', 'Alameda County',
       'Alpine County', 'Amador County', 'Butte County', 'Calaveras County',
       'Colusa County', 'Contra Costa County', 'Del Norte County',
       'El Dorado County', 'Fresno County', 'Glenn County', 'Humboldt County',
       'Imperial County', 'Inyo County', 'Kern County', 'Kings County',
       'Lake County', 'Lassen County', 'Los Angeles County', 'Madera County',
       'Marin County', 'Mariposa County', 'Mendocino County', 'Merced County',
       'Modoc County', 'Mono County', 'Monterey County', 'Napa County',
       'Nevada County', 'Orange County', 'Placer County', 'Plumas County',
       'Riverside County', 'Sacramento County', 'San Benito County',
       'San Bernardino County', 'San Diego County', 'San Francisco County',
       'San Joaquin County', 'San Luis Obispo County', 'San Mateo County',
       'Santa Barbara County', 'S

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [70]:
predictors=California_Severity_Data[['PercentCurrent','AreaCurrent','AreaMiles',
                                     'Year','Alameda County',
       'Alpine County', 'Amador County', 'Butte County', 'Calaveras County',
       'Colusa County', 'Contra Costa County', 'Del Norte County',
       'El Dorado County', 'Fresno County', 'Glenn County', 'Humboldt County',
       'Imperial County', 'Inyo County', 'Kern County', 'Kings County',
       'Lake County', 'Lassen County', 'Los Angeles County', 'Madera County',
       'Marin County', 'Mariposa County', 'Mendocino County', 'Merced County',
       'Modoc County', 'Mono County', 'Monterey County', 'Napa County',
       'Nevada County', 'Orange County', 'Placer County', 'Plumas County',
       'Riverside County', 'Sacramento County', 'San Benito County',
       'San Bernardino County', 'San Diego County', 'San Francisco County',
       'San Joaquin County', 'San Luis Obispo County', 'San Mateo County',
       'Santa Barbara County', 'Santa Clara County', 'Santa Cruz County',
       'Shasta County', 'Sierra County', 'Siskiyou County', 'Solano County',
       'Sonoma County', 'Stanislaus County', 'Sutter County', 'Tehama County',
       'Trinity County', 'Tulare County', 'Tuolumne County', 'Ventura County',
       'Yolo County', 'Yuba County']]
y=California_Severity_Data['USDMLevelID']

In [71]:
X_train, X_test, y_train, y_test = train_test_split(predictors,y, test_size=0.33)
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit_transform(X_train)
scaler.fit_transform(X_test)
logr = LogisticRegression()
logr.fit(X_train, y_train)
logr.score(X_train, y_train)
logr.score(X_test, y_test)

0.37159489274113144

In [72]:
California_Severity_Data['USDMLevelID'].value_counts()

1    34263
2    26680
3    18481
4     9321
5     5474
Name: USDMLevelID, dtype: int64

In [53]:
len(California_Severity_Data)

94219

In [73]:
34263/94219

0.3636527664271538

In [75]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
scores=cross_val_score(logr,X_train,y_train, cv=3)
scores
scores.mean()

0.36891280130753534

In [ ]:
California_Severity_Data